# MXNet

> Ejemplo adaptado del tutorial https://mxnet.apache.org/api/python/docs/tutorials/packages/gluon/image/mnist.html

Usamos la biblioteca MXNet, para lo cual hemos de instalarla previamente. En una instalación de Python de 64 bits, ejecutamos el siguiente comando:

`pip3 install --upgrade mxnet` 

Si disponemos de GPU en nuestro ordenador, podemos aprovecharla instalando la versión de MXNet con soporte para GPU:

`pip3 install --upgrade mxnet-cu101`

La versión indica que estamos instalando MXNet con CUDA/CuDNN usando la versión 10.1 de CUDA. Si en nuestro ordenador tenemos otra versión diferente de CUDA (p.ej. CUDA 9.2), tendremos que seleccionar la versión adecuada de MXNet (`mxnet-cu92`) tal como se indica en la web de MXNet: https://mxnet.io/get_started.

Cuando tengamos todo en orden, ya podemos utilizar MXNet desde Python:

In [1]:
import mxnet as mx

Empezamos con una comprobación de la versión que hemos instalado:

In [2]:
print (mx.__version__)

1.5.0


Si no se ha producido ningún error al ejecutar la línea anterior, podemos empezar a trabajar con MXNet en nuestro ordenador. Para que los experimentos sean reproducibles, podemos establecer la semilla del generador de números pseudoaleatorios...

In [3]:
mx.random.seed(42)  # Semilla del generador de números aleatorios

## El conjunto de datos MNIST

Descargamos el conjunto de datos MNIST:

In [4]:
mnist = mx.test_utils.get_mnist()

Y creamos dos iteradores para recorrer tanto el conjunto de entrenamiento como el conjunto de prueba:

In [5]:
batch_size = 128

train_data = mx.io.NDArrayIter(mnist['train_data'], mnist['train_label'], batch_size, shuffle=True)
val_data = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)

## Nuestra primera red neuronal en MXNet Gluon

Para nuestros experimentos con MXNet, utilizaremos el paquete Gluon, que proporciona un API de alto nivel:

In [6]:
from __future__ import print_function
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import autograd as ag

Definimos nuestra red neuronal multicapa:

In [12]:
# Definición de la red neuronal

net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(256, activation='relu'))
    net.add(nn.Dense(10))

Especificamos el contexto en el que ejecutaremos nuestro algoritmo (usando la GPU si está disponible):

In [13]:
gpus = mx.test_utils.list_gpus()
ctx =  [mx.gpu()] if gpus else [mx.cpu(0), mx.cpu(1)]

Inicializamos nuestra red neuronal y el algoritmo de optimización que vamos a utilizar:

In [14]:
net.initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

A continuación, entrenamos la red recorriendo una y otra vez el conjunto de entrenamiento:

In [15]:
%%time

epoch = 10

# Use Accuracy as the evaluation metric.
metric = mx.metric.Accuracy()
softmax_cross_entropy_loss = gluon.loss.SoftmaxCrossEntropyLoss()

for i in range(epoch):
    # Reset the train data iterator.
    train_data.reset()
    # Loop over the train data iterator.
    for batch in train_data:
        # Splits train data into multiple slices along batch_axis
        # and copy each slice into a context.
        data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
        # Splits train labels into multiple slices along batch_axis
        # and copy each slice into a context.
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        outputs = []
        # Inside training scope
        with ag.record():
            for x, y in zip(data, label):
                z = net(x)
                # Computes softmax cross entropy loss.
                loss = softmax_cross_entropy_loss(z, y)
                # Backpropagate the error for one iteration.
                loss.backward()
                outputs.append(z)
        # Updates internal evaluation
        metric.update(label, outputs)
        # Make one step of parameter update. Trainer needs to know the
        # batch size of data to normalize the gradient by 1/batch_size.
        trainer.step(batch.data[0].shape[0])
    # Gets the evaluation result.
    name, acc = metric.get()
    # Reset evaluation result to initial state.
    metric.reset()
    print('training acc at epoch %d: %s=%f'%(i, name, acc))
    

training acc at epoch 0: accuracy=0.876566
training acc at epoch 1: accuracy=0.927689
training acc at epoch 2: accuracy=0.943014
training acc at epoch 3: accuracy=0.951976
training acc at epoch 4: accuracy=0.959188
training acc at epoch 5: accuracy=0.964003
training acc at epoch 6: accuracy=0.968650
training acc at epoch 7: accuracy=0.971782
training acc at epoch 8: accuracy=0.974097
training acc at epoch 9: accuracy=0.976596
Wall time: 13.4 s


Tras unos segundos, obtenemos una red entrenada capaz de clasificar correctamente más del 90% de los ejemplos del conjunto de entrenamiento de MNIST. Sin embargo, lo que nos interesa es comprobar su capacidad predictiva sobre el conjunto de prueba:

In [16]:
# Use Accuracy as the evaluation metric.
metric = mx.metric.Accuracy()
# Reset the validation data iterator.
val_data.reset()

# Loop over the validation data iterator.
for batch in val_data:
    # Splits validation data into multiple slices along batch_axis
    # and copy each slice into a context.
    data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
    # Splits validation label into multiple slices along batch_axis
    # and copy each slice into a context.
    label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
    outputs = []
    for x in data:
        outputs.append(net(x))
    # Updates internal evaluation
    metric.update(label, outputs)

print('validation acc: %s=%f'%metric.get())
# assert metric.get()[1] > 0.94

validation acc: accuracy=0.972013


Si todo ha ido bien, habremos obtenido una red capaz de clasificar correctamente sobre el 97% de los ejemplos del conjunto de prueba de MNIST